In [209]:
import os
import sys
import hashlib

from shutil import rmtree,move,copy
from json import loads,dumps

import copy
import logger

In [210]:
class JSON(object):
    """
    Helper Class For Mapping JSON vars to Objects
    """
    def __init__(self,*args,**data):
        for key in data:
            if type(data[key]) == dict:
                self.__dict__[key] = JSON(**data[key])
            else:
                self.__dict__[key] = data[key]

    def __print__(self,level:int):
        indent = f'{((level+1)*4)*" "}'
        return "\n".join([
            f'{indent}{key}:{value.__repr__(level+1)}' 
            if   isinstance(value,JSON) 
            else f'{indent}{key}:{value.__repr__()}'
            
            for  key,value in self.__dict__.items()
        ])
    
    def __repr__(self,level=0):
        ret = ('JSON({\n' +
            self.__print__(level) +
        f'\n{(level*4)*" "}'+ '})')
        return ret
    
    def __str__(self):
        return self.__repr__()
    
    def __getitem__(self,key):
        return self.__dict__[key]
    
    def __iter__(self):
        for key in self.__dict__:
            if type(self.__dict__[key]) == JSON:
                yield key, self.__dict__[key]()
            else:
                yield key, self.__dict__[key]
    
    def __setitem__(self,key:str,value):
        self.__dict__[key] = value
        
    def __call__(self,):
        return {i:j for i,j in  self.__iter__() if not i.startswith("_")}
        
    def __delitem__(self,key):
        del self.__dict__[key]
                
    def __len__(self,)->int:
        return len(self())

    def __bool__(self,):
        return len(self) > 0

    def keys(self,):
        return self.__dict__.keys()


In [211]:
a = JSON(
    name="viraj",
    inner={
        "key":"value",
        "inner_2":{
            "key":"value"
        }
    }
)

In [212]:
a

JSON({
    name:'viraj'
    inner:JSON({
        key:'value'
        inner_2:JSON({
            key:'value'
        })
    })
})

In [213]:
class Record(JSON):
    def __init__(self,path:str=None,**kwargs):
        super().__init__(**kwargs)
        self._path = path

    def __str__(self):
        return self.__repr__()
        
    def write(self,):
        try:
            return f"Wrote {open(self._path,'w+').write(dumps(self()))} Chars Successfully"
        except:
            return f"Error Occured While Writing {self._path}"

    def destroy(self,):
        try:
            os.remove(self._path)
            return 1
        except:
            return 0

In [214]:
rec = Record(
    path="./test/rec.json",
    username="vptl185"
)

In [215]:
rec

JSON({
    username:'vptl185'
    _path:'./test/rec.json'
})

In [216]:
rec.write()

'Wrote 23 Chars Successfully'

In [241]:
class Table(object):
    def __init__(self,name,path):
        self._name = name
        self._path = path
        self._primary_key = open(os.path.join(path,"primary_key"),"r").read()
    
    def __repr__(self):
        return f"Table @ {self._path}"
    
    def __getitem__(self,key)->Record:
        path = os.path.join(self._path,f"{key}.json")
        if os.path.isfile(path):
            return Record(path)
        return JSON()
    
    def insertOne(self,record)->dict:
        assert self._primary_key in record.keys(),f"Record Does Not Contain Primary Key, In this case \"{self._primary_key}\""
        assert isinstance(record,Record), "Record should be of object of jsondb.Record"
        path = os.path.join(self._path,f"{record[self._primary_key]}.json")
        if os.path.isfile(path):
            return {
                "status":False,
                "message":f"{self._primary_key} Already Exists."
            }
        if open(path,'w+').write(dumps(record())):
            return {
                "status":True,
                "message":"Record Inserted Succesfully."
            }
        else:
            return {
                "status":False,
                "message":"Error Inserting Record.."
            }

    

    def deleteOne(self,key:str)->int:
        path = os.path.join(self._path,f"{key}.json")
        if os.path.isfile(path):
            try:
                os.remove(path)
                return 1
            except:
                return 2
        return 0
    
    def deleteAll(self,)->int:
        try:
            os.system(f"rm {self._path}/*.json")
            return 1
        except:
            return 0

    def updateOne(self,key:str,update:dict):
        pass

    def fetchKeys(self,)->list:
        return [i.split(".json")[0] for i in os.listdir(self._path) if i != 'primary_key']
    
    def fetchAll(self,start:int=0,end:int=-1)->list:
        return [
                loads(open(os.path.join(self._path,i)).read())
                for i in os.listdir(self._path) 
                if i.endswith(".json")
            ]

    def fetchOne(self,key:str)->Record:
        path = os.path.join(self._path,f"{key}.json")
        try:
            return Record(path=path,**loads(open(path,"r").read()))
        except:
            return Record()


In [242]:
class Database(object):
    def __init__(self,name:str,path:str,logger:logger.Logger=logger.Logger()):
        self._name = name
        self._path = os.path.join(path,name)
        self._logger = logger
        
        for table in os.listdir(self._path):
            if table not in query_keywords:
                self.__dict__[table] = Table(table,os.path.join(self._path,table))
        
    def __repr__(self):
        return f"Database @ {self._path}"
    
    def __getitem__(self,key)->Table:
        return self.__dict__[key]


    def listTables(self,):
        col,_ = os.get_terminal_size()
        print (col*'-')
        print (f"| Table{' '*12}| Path{' '*(col-26)}|")
        print (col*'-')
        for i in self.__dict__:
            if not i.startswith("_"):
                f = f"| {i}{(17-len(i))*' '}| {self.__dict__[i]}"
                print (f+(' '*(col-1-len(f)))+'|')
        print (col*'-')
    
    def createTable(self,name,primary_key):
        path = os.path.join(self._path,name)
        if os.path.isdir(path):
            self._logger.warning(f"Table {name} Already Exists.")
        else:
            os.mkdir(path)
            open(os.path.join(path,"primary_key"),"w+").write(primary_key)
            self.__dict__[name] = Table(name,path)
            self._logger.success(f"Table {name} Created Succesfully.")
    
    def deleteTable(self,name:str):
        path = os.path.join(self._path,name)
        if not os.path.isdir(path):
            self._logger.error(f"Table {name} Does Not Exist.")
        else:
            rmtree(path)
            del self.__dict__[name]
            self._logger.success(f"Table {name} Deleted Succesfully.")
    
    def renameTable(self,old:str,new:str):
        path = os.path.join(self._path,old)
        if not os.path.isdir(path):
            self._logger.error(f"Table {old} Does Not Exist.")
        else:
            path_ = os.path.join(self._path,new)
            move(path,path_)
            del self.__dict__[old]
            self.__dict__[new] = Table(new,path_)
            self._logger.success(f"Table Renamed Succesfully {old} -> {new}.")


In [243]:
class Cursor(object):
    """
    Cursor Class For JSONDB
    """
    def __init__(self,path:str="./",logger:logger.Logger=logger.Logger()):
        self._path = path
        self._db_path = os.path.join(self._path,"db")
        self._logger = logger
        
        if not os.path.isdir(self._db_path):
            os.mkdir(self._db_path)
            
        for db in os.listdir(self._db_path):
            self.__dict__[db] = Database(db,self._db_path,logger=logger)
            
    def __getitem__(self,key)->Database:
        return self.__dict__[key]

    def __repr__(self)->str:
        return f"Connection @ {self._path}"
            
    def createDB(self,name:str):
        path = os.path.join(self._path,"db",name)
        if not os.path.isdir(path):
            os.mkdir(path)
            self.__dict__[name] = Database(name,self._db_path)
            self._logger.success(f"Database {name} Created Succesfully.")
        else:
            self._logger.error(f"Database {name} Already Exists.")

    def listDB(self,):
        col,_ = os.get_terminal_size()
        print (col*'-')
        print (f"| Database{' '*9}| Path{' '*(col-26)}|")
        print (col*'-')
        for i in self.__dict__:
            if not i.startswith("_"):
                f = f"| {i}{(17-len(i))*' '}| {self.__dict__[i]}"
                print (f+(' '*(col-1-len(f)))+'|')
        print (col*'-')

    def deleteDB(self,name:str):
        path = os.path.join(self._path,"db",name)
        if not os.path.isdir(path):
            self._logger.error(f"Database {name} Does Not Exist.")
        else:
            rmtree(path)
            del self.__dict__[name]
            self._logger.success(f"Database {name} Deleted Succesfully.")


def connect(path:str="./")->Cursor:
    path = os.path.abspath(path)
    if not os.path.isdir(path):
        os.mkdir(path=path)
        open(os.path.join(path,"config.json"),"w+")
    return Cursor(path=path)


In [244]:
conn = connect(path="./test/db")

In [245]:
conn

Connection @ /home/codekage/workspace/transfer-pi-dev/jsondb/test/db

In [246]:
conn.createDB("USERS")

 [SUC] Database USERS Created Succesfully.                                                                                                                                                                                                          [ 2020-11-07 23:56:04 ]


In [249]:
conn.USERS.createTable("user","email")

 [SUC] Table user Created Succesfully.                                                                                                                                                                                                              [ 2020-11-07 23:56:30 ]


In [252]:
conn.USERS.user.insertOne(Record(
    email="vptl185",
    val="test"
))

{'status': False, 'message': 'email Already Exists.'}

In [254]:
users = conn.USERS.user